# Exploratory Data Analysis
This notebook is used for exploring the hotdog/nothotdog image dataset.

In [2]:
import os
import glob
import PIL.Image as Image

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm 

Provided class to load the data.

In [3]:
class Hotdog_NotHotdog(torch.utils.data.Dataset):
    def __init__(self, train=True, transform=None, data_path='hotdog_nothotdog', processor=None):
        self.transform = transform
        self.processor = processor
        data_path = os.path.join(data_path, 'train' if train else 'test')
        
        # Get class names
        image_classes = [os.path.split(d)[1] for d in glob.glob(data_path + '/*') if os.path.isdir(d)]
        image_classes.sort()
        self.name_to_label = {c: idx for idx, c in enumerate(image_classes)}
        
        # Get image paths
        self.image_paths = glob.glob(data_path + '/*/*.jpg')

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        # Apply processor normalization if given
        if self.processor:
            # processor expects PIL Image, but we already applied transforms.
            # If using transforms.ToTensor(), you can manually normalize
            # Here, we convert to a tensor and normalize
            image = self.processor(image, return_tensors="pt")["pixel_values"].squeeze(0)

        # Get label
        class_name = os.path.split(os.path.split(image_path)[0])[1]
        label = self.name_to_label[class_name]

        return {"pixel_values": image, "labels": label}

Loading the data. Transforming the image to tensors but applying no resizing, thus keeping the original dimensions of the images.

In [4]:
size = 224

# resizing, cropping, and flipping
train_transform = transforms.Compose([
    transforms.ToTensor(),
])

test_transform = transforms.Compose([                      
    transforms.ToTensor(),                 
])

batch_size = 64
trainset = Hotdog_NotHotdog(train=True, transform=train_transform, data_path='hotdog_nothotdog')
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
testset = Hotdog_NotHotdog(train=False, transform=test_transform, data_path='hotdog_nothotdog')
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

In [5]:
n_train = len(trainset)
n_test = len(testset)

min_shape_x = 1000
max_shape_x = 1
min_shape_y = 1000
max_shape_y = 1
min_pixels = 1e20
max_pixels = 1

for i in tqdm(range(n_train)):
    n_feat, x, y = trainset[i]["pixel_values"].shape
    n_pixels = x*y 

    if n_pixels < min_pixels:
        min_pixels = n_pixels 
        min_shape_x = x 
        min_shape_y = y 
    
    if n_pixels > max_pixels:
        max_pixels = n_pixels 
        max_shape_x = x 
        max_shape_y = y 

for i in tqdm(range(n_train)):
    n_feat, x, y = trainset[i]["pixel_values"].shape
    n_pixels = x*y 

    if n_pixels < min_pixels:
        min_pixels = n_pixels 
        min_shape_x = x 
        min_shape_y = y 
    
    if n_pixels > max_pixels:
        max_pixels = n_pixels 
        max_shape_x = x 
        max_shape_y = y 

print(f"The smallest image has size {min_shape_x} x {min_shape_y}")
print(f"The largest image has size {max_shape_x} x {max_shape_y}")

100%|██████████| 2047/2047 [00:08<00:00, 254.61it/s]

The smallest image has size 61 x 61
The largest image has size 2100 x 3150
